<a href="https://colab.research.google.com/github/plaban1981/Generative_AI/blob/main/GPT4ALLV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 8 bit implmentation

In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install datasets sentencepiece 
!pip -q install bitsandbytes accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 688.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 31

## GPT4ALLv2 GPT-J 6B

In [2]:
! nvidia-smi

Sat Apr 15 15:43:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
from transformers import pipeline
#
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("nomic-ai/gpt4all-j")

model = AutoModelForCausalLM.from_pretrained("nomic-ai/gpt4all-j", 
                                             load_in_8bit=True,
                                             device_map="auto")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
from transformers import  GenerationConfig, pipeline
import torch
import textwrap
generate_text = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    #max_length=512,
    max_new_tokens=1000,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
    )
def gpt4all_generate(text):
  print("Generating ....")
  generated_text = generate_text(text)
  text = generated_text[0]['generated_text']
  # Split the input text into lines based on newline characters
  lines = text.split('\n')

  # Wrap each line individually
  wrapped_lines = [textwrap.fill(line, width=110) for line in lines]

  # Join the wrapped lines back together using newline characters
  wrapped_text = '\n'.join(wrapped_lines)

  return wrapped_text


In [24]:

%%time
PROMPT="""You are a helpful healthcare assistant. Ask me anything about your medication regimen and I will provide real-time information about drugs, including side effects, interactions, and potential contraindications. Simply tell me the name of your medication and I will do my best to provide you with accurate and timely information to help you make informed decisions about your health.
###INSTRUCTION:
Please provide the details for all the medicines prescribed in a point-by-point manner:
1. Provide General information on the medications you listed in a point wise manner. 
2. Provide Real-time information about drugs, including side effects, interactions, and potential contra-indications. 
3. Provide Proper dosage, administration, and storage of medications
4. Suggest Potential alternatives for patients who are allergic or intolerant to specific prescriptions if any.
5. Also correct the medication names if specified incorrectly.

The output should be generated in Markdown Format.Make sure details should be h2 tag & response to the details should be p tag

MEDICINES PRESCRIBED:
Medicines Prescribed:  
1. Glucobay 50 mg
2. Reclide 80 mg
3. Glycomet  25 mg
4. Glucobay 50 mg
5. Fourtiz B Z Plus
Allergic/ Intolerant:  
Metmorfin
###RESPONSE:
"""
out = gpt4all_generate(PROMPT)
     

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generating ....
CPU times: user 20.2 s, sys: 11.5 ms, total: 20.2 s
Wall time: 20.1 s


In [25]:
print(out)

You are a helpful healthcare assistant. Ask me anything about your medication regimen and I will provide real-
time information about drugs, including side effects, interactions, and potential contraindications. Simply
tell me the name of your medication and I will do my best to provide you with accurate and timely information
to help you make informed decisions about your health.
###INSTRUCTION:
Please provide the details for all the medicines prescribed in a point-by-point manner:
1. Provide General information on the medications you listed in a point wise manner.
2. Provide Real-time information about drugs, including side effects, interactions, and potential contra-
indications.
3. Provide Proper dosage, administration, and storage of medications
4. Suggest Potential alternatives for patients who are allergic or intolerant to specific prescriptions if
any.
5. Also correct the medication names if specified incorrectly.

The output should be generated in Markdown Format.Make sure det

In [32]:
%%time 
out = gpt4all_generate('Write a short note to Sam Altman giving reasons to open source GPT-4.\n ####RESPONSE :\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generating ....
CPU times: user 34 s, sys: 37.6 ms, total: 34.1 s
Wall time: 34.2 s


In [33]:
out

"Write a short note to Sam Altman giving reasons to open source GPT-4.\n ####RESPONSE :\nAs an AI language model, I don't have personal opinions or beliefs but here are some possible responses for\nSam Altman's reason to open source GPT-4:\n\n 1. To make it easier and more accessible for researchers and developers who need advanced natural language\nprocessing capabilities in their work - providing the code as open source will allow anyone to learn from it\nand potentially contribute improvements that can be incorporated into future versions of the software. This\ncould lead to faster development cycles, better performance on specific tasks, and overall greater efficiency\nin fields like machine translation, speech recognition, etc. Additionally, by sharing the underlying\nalgorithms used in GPT-4 with others in the community, we may discover new ways to optimize them further or\naddress any issues identified through feedback."

In [30]:
%%time 
out = gpt4all_generate('What is the capital of England? \n ####RESPONSE :\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generating ....
CPU times: user 2.12 s, sys: 4.46 ms, total: 2.13 s
Wall time: 2.12 s


In [31]:
out

'What is the capital of England?\n ####RESPONSE :\nThe capital city of England is London.'

In [34]:
%%time
out = gpt4all_generate('Write a poem about Large Language Models in the style of Shakespeare.\n####RESPONSE:\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generating ....
CPU times: user 27.4 s, sys: 44.5 ms, total: 27.5 s
Wall time: 27.5 s


In [35]:
out

"Write a poem about Large Language Models in the style of Shakespeare.\n####RESPONSE:\nOh, fair maiden who doth grace this stage with her beauty and wit\nMay I presume thee to be the one that hath caused such a stir?\nFor thou hast captured the hearts of all gathered here tonight\nAnd thy words have stirred our souls like nothing seen before\nThou speaketh eloquent as canst any tongue known to man\nAs though it were spoken by angels from heaven's own hand\nBut mayhap it is thy voice alone which brings forth such delight\nThat we forget for just a moment time itself takes flight!"

In [36]:
%%time
PROMPT = """You are a helpful Coding assistant. Ask me anything about coding logic in any programming language and I will provide the code for the logic requested:
Write a python code to detect if a give string is palindrome or not.  
CODE GENERATED:

"""
out = gpt4all_generate(PROMPT)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generating ....
CPU times: user 1min 6s, sys: 86.6 ms, total: 1min 6s
Wall time: 1min 6s


In [38]:
print(out.split("CODE GENERATED")[-1])

:


```python
def check_palindrome(string):
    # Get length of each word
    words = list(string)
    len1, len2 = len(words), len(set(word))
    print("Lengths : ", len1, len2)

    # Check every letter against its corresponding position in reverse order
    for i in range(len1-1,-1):
        j = len2 - 1 - i
        while True:
            try:
                if words[i]!= set(reverse(words)).pop():
                    return False
            except IndexError:
                break
        else:
            return True

    # If all letters match, then it's a palindrome
    return True


In [54]:
%%time
PROMPT = """Create a set of 5 multiple choice quiz on the topic specified below and provide answers.
TOPIC:
<<topic>>
####RESPONSE:
"""
topic_text = "Symbiosis"
PROMPT = PROMPT.replace("<<topic>>",topic_text)
out = gpt4all_generate(PROMPT)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generating ....
CPU times: user 19.4 s, sys: 32.4 ms, total: 19.4 s
Wall time: 19.4 s


In [52]:
out

"Create a set of 5 multiple choice quiz on the topic specified below and provide answers.\n####RESPONSE:\nSorry, as an AI language model I don't have any context about which topic is being asked in this prompt.\nPlease specify your question so that I can assist you better."

In [55]:
print(out.split("####RESPONSE:")[-1])


1. How does symbiosis benefit both partners?
Answer: Symbiosis is beneficial for both partners because it allows them to share resources, such as food or
shelter, without giving up their own needs. This mutual dependence leads to greater cooperation between
organisms that would not be possible if one partner were completely self-sufficient. Additionally, some
species rely heavily on symbiotic relationships with other members of their ecosystem in order to survive and
reproduce successfully.


In [64]:
%%time
prompt=""" Generate TRUE or FALSE statements based on the context provided as below:
Context: The biproduct of photosynthesis process is oxygen.Increased light intensity results in a higher rate of photosynthesis. On the other hand, low light intensity results in a lower rate of photosynthesis. Higher concentration of carbon dioxide helps in increasing the rate of photosynthesis. Usually, carbon dioxide in the range of 300 – 400 PPM is adequate for photosynthesis
Statement: Photosynthesis is a process that results in the production of oxygen 
Answer: True
Statement: Carbon dioxide in the range of 300 – 400 PPM is required for photosynthesis. 
Answer: True
Statement:  Photosynthesis process does not produce oxygen as a biproduct.  
Answer: False
###
Context: NLP Cloud developed their API by mid-2020 and they added many pre-trained open-source models since then.
Statement: NLP Cloud developed their API in mid-2020. 
Answer: True
Statement: NLP Cloud only added one pre trained model since then. 
Answer: False
###
Context: Context : Google was founded in 1998 by larry page and sergey brin while they were ph.d. students at stanford university in california. In august 2015, google announced plans to reorganize its various interests as a conglomerate called alphabet inc. Google is alphabet's leading subsidiary and will continue to be the umbrella company for alphabet's internet interests. Sundar pichai was appointed ceo of google.
####RESPONSE:

"""

out = gpt4all_generate(prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generating ....
CPU times: user 13.1 s, sys: 20.1 ms, total: 13.2 s
Wall time: 13.2 s


In [65]:
print(out.split("####RESPONSE:")[-1])



 Statement: Alphabet Inc. formed a new holding company named “Alphabet” with Larry Page becoming CEO after
Sergey Brin stepped down from his position due to health issues in 2018.
 Answer: True
